# cuaderno colab de entrenamiento de `Flowtron`(en español)
Cuaderno hecho por [rmcpantoja](https://github.com/rmcpantoja)
# créditos:
* Este cuaderno está basado en el [Cuaderno original](https://colab.research.google.com/drive/1B0DLBIcqOejPr_syAzOfeK9Ib1WE78Gq) hecho por `reubenkway`.

* [repositorio de nvidia/flowtron](http://github.com/nvidia/flowtron)


## configuración:
Aquí está todo lo necesario para la instalación del sintetizador y sus dependencias. Recuerda ejecutar las celdas en orden para que funcione correctamente.

In [ ]:
#@markdown ### 1: comprobar GPU
!nvidia-smi -L

In [ ]:
#@markdown ### 2: instalación y configuración de herramientas
#@markdown ---
#@markdown Montaremos tu drive para trabajar con un conjunto de datos e instalaremos las herramientas necesarias.
#@markdown * Si se te pregunta si quieres conectar con google drive, haz click en el botón correspondiente.
#@markdown * Al finalizar, es provable que se te pida `reiniciar el entorno`. Es muy recomendable hacerlo antes de proceder con las siguientes celdas.
from google.colab import drive
drive.mount('/content/drive')
%cd /content/
!git clone https://github.com/NVIDIA/flowtron.git
%cd /content/flowtron
!git submodule update --init 
%cd /content/flowtron/tacotron2 
!git submodule update --init
%cd /content/flowtron
!pip install -r requirements.txt

In [ ]:
#@markdown ### 2.1: instalación y configuración de herramientas - parte dos
#@markdown ---
#@markdown Ejecuta esta celda para finalizar la configuración.
!sed -i -- 's,from numba.decorators import jit as optional_jit, ,g' '/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py'

## entrenamiento:
Esta sección es la parte del entrenamiento, donde puedes establecer los parámetros, rutas y configuraciones para el modelo.

In [ ]:
#@markdown ### 3: configuraciones
#@markdown ---
#@markdown Aquí puedes configurar la ruta para los audios y transcripciones, así como opciones para el entrenamiento.
!mkdir /content/flowtron/wavs
#@markdown #### dale un nombre al modelo
model_name = "auronplay"h
#@markdown ---
#@markdown #### directorio de salida
output_directory = "/content/drive/MyDrive/flowtron_checkpoints" #@param {type:"string"}
#@markdown ---
#@markdown #### la ruta del zip de los audios
WavsPath = "/content/drive/MyDrive/Fakeyou/auronflowtron}/wavs.zip" #@param {type:"string"}
!unzip $WavsPath -d /content/flowtron/wavs
#@markdown ---
#@markdown #### rutas de transcripción
#@markdown * El archivo de validación es el mismo que insertaremos en el archivo de entrenamiento.
#@markdown * A diferencia del nativo tacotron 2 single speaker, necesitamos un tercer parámetro para las listas, que será la id. Del hablante. Esto biene bien para hacer un modelo multivoz. A continuación, un ejemplo:
#@markdown  * wavs/1.wav|texto|0
#@markdown  * wavs/1b.wav|texto segunda voz|1
training_files = "/content/drive/MyDrive/Fakeyou/auronflowtron}/list.txt" #@param {type:"string"}
!sed -i -- 's,"training_files": "filelists/ljs_audiopaths_text_sid_train_filelist.txt", "training_files": "{training_files}",g' '/content/flowtron/config.json'
validation_files = "/content/drive/MyDrive/Fakeyou/auronflowtron}/list.txt" #@param {type:"string"}
!sed -i -- 's,"validation_files": "filelists/ljs_audiopaths_text_sid_val_filelist.txt", "validation_files": "{validation_files}",g' '/content/flowtron/config.json'
#@markdown ---
#@markdown #### la taza de aprendizaje
learning_rate = 0.0005 #@param {type:"number"}
!sed -i -- 's,"learning_rate": 0.001, "learning_rate": {learning_rate},g' '/content/flowtron/config.json'
#@markdown ---
#@markdown #### elegir tamaño de lote
#@markdown ---
#@markdown un lote más alto puede llegar a consumir más memoria ram, así que por favor elige con precaución.
batch_size = 2 #@param {type:"integer"}
!sed -i -- 's,"batch_size": 6, "batch_size": {batch_size},g' '/content/flowtron/config.json'
#@markdown ---
#@markdown #### épocas para entrenar
epochs = 500 #@param {type:"integer"}
!sed -i -- 's,"epochs": 10000000, "epochs": {epochs},g' '/content/flowtron/config.json'
#@markdown ---
#@markdown #### iteraciones por punto de control (no se recomienda cambiar)
iters_per_checkpoint = 25 #@param {type:"integer"}
!sed -i -- 's,"iters_per_checkpoint": 1000, "iters_per_checkpoint": {iters_per_checkpoint},g' '/content/flowtron/config.json'
#@markdown ---
#@markdown #### Elegir número de hablantes
n_speakers = 1 #@param {type:"integer"}
!sed -i -- 's,"n_speakers": 1, "n_speakers": {n_speakers},g' '/content/flowtron/config.json'
#@markdown ---
# set flowtron to basic:
!sed -i -- 's,"text_cleaners": "flowtron_cleaners", "text_cleaners": "basic_cleaners",g' '/content/flowtron/config.json'

In [ ]:
#@markdown ### 4: entrenar
#@markdown ---
#@markdown ejecuta esta celda para comenzar el entrenamiento del modelo.
#@markdown * Puedes activar o desactivar el tensorboard en la casilla que se te muestra más abajo.
#@markdown  * El tensorboard sirve para visualizar el proceso de entrenamiento del modelo.
#@markdown * ¡Nota! Si cuda se queda sin memoria, vé a la celda de **configuración**, disminuye el tamaño de lote y ejecútala.
#@markdown ---
#@markdown falta por hacer: arreglar el entrenamiento con tensorboard.
run_tensorboard = True #@param {type:"boolean"}
if run_tensorboard:
  %load_ext tensorboard
  import tensorflow as tf
  import datetime, os
  %tensorboard --logdir '$output_directory/logs'
%cd /content/flowtron/
print("Descarga del modelo preentrenado...")
!gdown https://drive.google.com/uc?id=1Cjd6dK_eFz6DE0PKXKgKxrzTUqzzUDW-
!sed -i -- 's,model_{},{model_name},g' '/content/flowtron/train.py'
! python3 train.py -c config.json -p train_config.learning_rate=0.0001 train_config.warmstart_checkpoint_path="/content/flowtron/flowtron_ljs.pt" train_config.output_directory="$output_directory" train_config.with_tensorboard=$run_tensorboard